#### tf.control_dependencies(control_inputs) 指定操作依赖关系
此函数指定某些操作执行的依赖关系, 返回一个控制依赖的上下文管理器，使用 with 关键字可以让在这个上下文环境中的操作都在 control_inputs 执行    
````
with tf.control_dependencies([a, b]):
    c = ....
    d = ...
````
在执行完 a，b 操作之后，才能执行 c，d 操作。意思就是 c，d 操作依赖 a，b 操作    
````
with tf.control_dependencies([train_step, variable_averages_op]):
    train_op = tf.no_op(name='train')
```
tf.no_op()表示执行完 train_step, variable_averages_op 操作之后什么都不做 

##### 可以传入None 来消除依赖：
```
with g.control_dependencies([a, b]):
  # Ops constructed here run after `a` and `b`.
  with g.control_dependencies(None):
    # Ops constructed here run normally, not waiting for either `a` or `b`.
    with g.control_dependencies([c, d]):
      # Ops constructed here run after `c` and `d`, also not waiting
      # for either `a` or `b`
```

控制依赖只对那些在上下文环境中建立的操作有效，仅仅在context中使用一个操作或张量是没用的    
```
# WRONG
def my_func(pred, tensor):
  t = tf.matmul(tensor, tensor)
  with tf.control_dependencies([pred]):
    # The matmul op is created outside the context, so no control
    # dependency will be added.
    return t

# RIGHT
def my_func(pred, tensor):
  with tf.control_dependencies([pred]):
    # The matmul op is created in the context, so a control dependency
    # will be added.
    return tf.matmul(tensor, tensor)
```

#### tf.group()用于组合多个操作
用于创造一个操作，可以将传入参数的所有操作进行分组。API手册如:    
```
tf.group(
    *inputs,
    **kwargs
)
ops = tf.group(tensor1, tensor2,...) 
```
其中*inputs是0个或者多个用于组合tensor，一旦ops完成了，那么传入的tensor1,tensor2,...等等都会完成了，经常用于组合一些训练节点，如在Cycle GAN中的多个训练节点，例子如：
```
generator_train_op = tf.train.AdamOptimizer(g_loss, ...)
discriminator_train_op = tf.train.AdamOptimizer(d_loss,...)
train_ops = tf.groups(generator_train_op ,discriminator_train_op)

with tf.Session() as sess:
  sess.run(train_ops) 
  # 一旦运行了train_ops,那么里面的generator_train_op和discriminator_train_op都将被调用
```

##### 注意的是，tf.group()返回的是个操作，而不是值，如果你想下面一样用，返回的将不是值
```
a = tf.Variable([5])
b = tf.Variable([6])
c = a+b
d = a*b
e = a/b
ops = tf.group(c,d,e)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ee = sess.run(ops)
```
返回的将不是c,d,e的运算结果，而是一个None，就是因为这个是一个操作，而不是一个张量。如果需要返回结果，请参考tf.tuple()
##### 具体例子看后面

#### [tf.group/tf.tuple 区别](https://blog.csdn.net/liuweiyuxiang/article/details/79953259?utm_medium=distribute.pc_relevant_t0.none-task-blog-BlogCommendFromMachineLearnPai2-1.nonecase&depth_1-utm_source=distribute.pc_relevant_t0.none-task-blog-BlogCommendFromMachineLearnPai2-1.nonecase)
```
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, first_clone_scope)
......
update_op = tf.group(*update_ops)
```
group是分组的意思，其实该函数就是把update_ops（list）中的操作作为一个组，把这些操作和成为一个操作。除了group函数之外，tuple函数也有类似的功能，但是有一点细微的差别。
1. 首先是接受参数的形式不同。
group的参数是一个一个operation，而不是一个列表。这也是引言中的update_ops列表前需要加*的原因。
1. 返回值不同。
tf.group()返回的是op，tf.tuple()返回的是list of tensor。
如果还是不太理解的话可以看后面的例子，和下面的英文的api，英文的内容比较简单，我就不翻译了。
* 可以发现，如果我们有很多 tensor 或 op想要一起run，tf.group() 与 tf.tuple()两个函数就是一个很好的帮手了。*


In [4]:
import tensorflow as tf

w = tf.Variable(1)
mul = tf.multiply(w, 2)
add = tf.add(w, 2)
group = tf.group(mul, add)
tuple = tf.tuple([mul, add])
# sess.run(group)和sess.run(tuple)都会求Tensor(add)
#Tensor(mul)的值。区别是，tf.group()返回的是`op`
#tf.tuple()返回的是list of tensor。
#这样就会导致，sess.run(tuple)的时候，会返回 Tensor(mul),Tensor(add)的值.
#而 sess.run(group)不会

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print("group:", sess.run(group))
    print("tuple:", sess.run(tuple))

group: None
tuple: [2, 3]


#### tf.identity()
在学习tf.group()的时候，看到很多文章都是将tf.identity()和tf.group()放在一起辨析，就一起学习了。字面上来理解identity是恒等的意思，其实这就是一个赋值操作。在一般的情况下，我们使用赋值操作符=来进行赋值，例如y=x,表示将x的值赋值给y。但是在TensorFlow某些特殊的情况下是不支持这么做的，原因是TensorFlow中的计算都是基于计算图中的，计算图的每个节点都是一个operation对象，所有以上的赋值操作也需要用一个赋值操作来表示，才能在计算图中进行计算(赋值)。所以需要写成y=tf.identity(x)。


In [4]:
import tensorflow as tf

x = tf.Variable(0.0)
#返回一个op，表示给变量x加1的操作
x_plus_1 = tf.assign_add(x, 1)

#control_dependencies的意义是，在执行with包含的内容（在这里就是 y = x）前
#先执行control_dependencies中的内容（在这里就是 x_plus_1）
with tf.control_dependencies([x_plus_1]):
    y = x

print("leotest")
with tf.Session() as sess:
    print("what's wrong")
    tf.global_variables_initializer().run()
    print("into range:")
    for i in range(5):
        print(sess.run(y))#相当于sess.run(y)，由于control_dependencies的所以执行print前都会先执行x_plus_1


leotest
what's wrong
into range:
0.0
0.0
0.0
0.0
0.0


In [6]:
import tensorflow as tf

x = tf.Variable(0.0)
#返回一个op，表示给变量x加1的操作
x_plus_1 = tf.assign_add(x, 1)

#control_dependencies的意义是，在执行with包含的内容（在这里就是 y = x）前
#先执行control_dependencies中的内容（在这里就是 x_plus_1）
with tf.control_dependencies([x_plus_1]):
    y = tf.identity(x)

print("leotest")
with tf.Session() as sess:
    print("what's wrong")
    tf.global_variables_initializer().run()
    print("into range:")
    for i in range(5):
        print(sess.run(y))#相当于sess.run(y)，由于control_dependencies的所以执行print前都会先执行x_plus_1

leotest
what's wrong
into range:
1.0
2.0
3.0
4.0
5.0
